In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls

autoencoder8.h5      hdf5				phaseTwo_ImageNet.ipynb
embeddings_list.npy  phaseOne_Autoencoder.ipynb		Untitled0.ipynb
embeddings.npy	     phaseTwo_generateEmbeddings.ipynb


In [0]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
!pwd

/content/drive/My Drive


In [0]:
!ls

autoencoder8.h5      hdf5				phaseTwo_ImageNet.ipynb
embeddings_list.npy  phaseOne_Autoencoder.ipynb		Untitled0.ipynb
embeddings.npy	     phaseTwo_generateEmbeddings.ipynb


In [0]:
!pip install -q keras

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12201200498031237678, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8316662964547154205
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7532586288982232412
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13322309180961756119
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# **Autoencoder**

## load trained autoencoder

In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [0]:
import numpy as np
import pandas as pd


from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Conv3D, Conv3DTranspose, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D, Input, BatchNormalization, Dropout, Lambda, Bidirectional
from keras.layers import CuDNNLSTM, Reshape, PReLU, ZeroPadding2D
from keras.layers.merge import concatenate
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import load_model
from keras.callbacks import Callback
from keras.utils import plot_model

from keras.preprocessing import image
import keras.backend as K

import matplotlib.pyplot as plt

import time 
import tensorflow as tf
from sklearn import metrics
import pickle
import os
import scipy.io as sio

Using TensorFlow backend.


In [0]:
import PIL

In [0]:

np.random.seed(5)
tf.set_random_seed(5)

In [0]:
from keras.applications.vgg19 import VGG19, decode_predictions, preprocess_input

In [0]:
import h5py

In [0]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [0]:
#load trained model

autoencoder_trained = load_model("autoencoder8.h5", custom_objects = {"precision":precision})
autoencoder_trained.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ac (InputLayer)        (None, 20, 20, 20, 1)     0         
_________________________________________________________________
conv1_ac (Conv3D)            (None, 14, 14, 14, 96)    33024     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 14, 14, 14, 96)    263424    
_________________________________________________________________
conv2_ac (Conv3D)            (None, 10, 10, 10, 256)   3072256   
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 10, 10, 10, 256)   256000    
_________________________________________________________________
conv3_ac (Conv3D)            (None, 8, 8, 8, 384)      2654592   
_________________________________________________________________
p_re_lu_3 (PReLU)            (None, 8, 8, 8, 384)      196608    
__________

# **Encoder**

## izdvajanje samo prvog dijela treniranog autoencodera, tj. izdvajanje encodera

In [0]:
# definiranje Encodera, iz prvog dijela treniranog Autoencoder

def Encoder(autoencoder_model):
    
    input_vox = Input(shape=(20,20,20,1,), name = "input_ac")
    conv1 = Conv3D(96, 7, name = "conv1_ac")(input_vox)
    act1 = PReLU(name = "p_re_lu_1")(conv1)
    conv2 = Conv3D(256, 5, name = "conv2_ac")(act1)
    act2 = PReLU(name = "p_re_lu_2")(conv2)
    conv3 = Conv3D(384, 3, name = "conv3_ac")(act2)
    act3 = PReLU(name = "p_re_lu_3")(conv3)
    conv4 = Conv3D(256, 3, name = "conv4_ac")(act3)
    act4 = PReLU(name = "p_re_lu_4")(conv4)

    flat = Flatten(name = "flat_ac")(act4)
    out_embedding = Dense(64, name = "dense1_ac")(flat)
    
    encoder = Model(inputs = input_vox, outputs = out_embedding)
    
    print("Učitavanje težina...")
    for enc_layer in encoder.layers:
        try:
            enc_layer.set_weights(autoencoder_model.get_layer(enc_layer.name).get_weights())
        except ValueError:
            print("Nema layera: ", enc_layer.name)
            
        
    print("Kontrola težina...")
    for enc_layer in encoder.layers:
        try:
            print(enc_layer.name)
            if len(autoencoder_model.get_layer(enc_layer.name).get_weights()) == 0: 
                print("Nista.")
                continue
            print((autoencoder_model.get_layer(enc_layer.name).get_weights()[0] == encoder.get_layer(enc_layer.name).get_weights()[0]).all())
            if len(autoencoder_model.get_layer(enc_layer.name).get_weights()) > 1:
                print((autoencoder_model.get_layer(enc_layer.name).get_weights()[1] == encoder.get_layer(enc_layer.name).get_weights()[1]).all())
        except ValueError:
            print("Nema layera ", enc_layer.name)
    return encoder

In [0]:
encoder = Encoder(autoencoder_trained)

Učitavanje težina...
Kontrola težina...
input_ac
Nista.
conv1_ac
True
True
p_re_lu_1
True
conv2_ac
True
True
p_re_lu_2
True
conv3_ac
True
True
p_re_lu_3
True
conv4_ac
True
True
p_re_lu_4
True
flat_ac
Nista.
dense1_ac
True
True


In [0]:
len(os.listdir("/content/drive/My Drive/hdf5"))

101

In [0]:
listdir = os.listdir("/content/drive/My Drive/hdf5")

import re
def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)
  
listdir_sorted = sorted_aphanumeric(listdir)
listdir_sorted

['batch_0.h5',
 'batch_1.h5',
 'batch_2.h5',
 'batch_3.h5',
 'batch_4.h5',
 'batch_5.h5',
 'batch_6.h5',
 'batch_7.h5',
 'batch_8.h5',
 'batch_9.h5',
 'batch_10.h5',
 'batch_11.h5',
 'batch_12.h5',
 'batch_13.h5',
 'batch_14.h5',
 'batch_15.h5',
 'batch_16.h5',
 'batch_17.h5',
 'batch_18.h5',
 'batch_19.h5',
 'batch_20.h5',
 'batch_21.h5',
 'batch_22.h5',
 'batch_23.h5',
 'batch_24.h5',
 'batch_25.h5',
 'batch_26.h5',
 'batch_27.h5',
 'batch_28.h5',
 'batch_29.h5',
 'batch_30.h5',
 'batch_31.h5',
 'batch_32.h5',
 'batch_33.h5',
 'batch_34.h5',
 'batch_35.h5',
 'batch_36.h5',
 'batch_37.h5',
 'batch_38.h5',
 'batch_39.h5',
 'batch_40.h5',
 'batch_41.h5',
 'batch_42.h5',
 'batch_43.h5',
 'batch_44.h5',
 'batch_45.h5',
 'batch_46.h5',
 'batch_47.h5',
 'batch_48.h5',
 'batch_49.h5',
 'batch_50.h5',
 'batch_51.h5',
 'batch_52.h5',
 'batch_53.h5',
 'batch_54.h5',
 'batch_55.h5',
 'batch_56.h5',
 'batch_57.h5',
 'batch_58.h5',
 'batch_59.h5',
 'batch_60.h5',
 'batch_61.h5',
 'batch_62.h5',
 '

In [0]:
def generate_batches_embedding(filepath):
  while True:
    for batch_file in listdir_sorted:
      f = h5py.File(filepath+"/"+batch_file)
      #images = f['data'].value
      #images = np.swapaxes(images, 1, 3) # swap axis, channel should be last
      voxels = f['label-voxel'].value
      voxels = np.swapaxes(voxels, 1, 4)
      
      yield voxels

In [0]:
# prediktanje pomocu generatora, problem sto sve stavi u isti array...
# sejvano u AC_embeddings.npy
embeddings = encoder.predict_generator(generate_batches_embedding("/content/drive/My Drive/hdf5"), steps = 101, use_multiprocessing = True, verbose = 1)
embeddings.shape

101/101 [==============================] - 79s 779ms/step


(20056, 64)

In [0]:
embeddings

array([[-11.7458935 , -10.152479  , -14.98231   , ..., -14.835023  ,
          4.306768  , -15.785215  ],
       [  1.7940768 ,   0.5956671 , -11.05381   , ...,  -2.6179147 ,
         -1.036053  , -12.755989  ],
       [-18.71303   , -22.291632  ,  -7.1848354 , ..., -15.208323  ,
         -4.3542786 ,  -6.375044  ],
       ...,
       [-16.29517   , -23.261217  ,  -3.1317303 , ..., -16.611673  ,
          3.3800707 ,  -4.042325  ],
       [-12.140161  ,  -5.451047  , -11.566293  , ...,  -8.997949  ,
         -0.12947914,  -8.149326  ],
       [-12.962855  ,  -2.3739915 ,   6.7128553 , ...,  -1.8349651 ,
          3.5588815 ,  -7.388445  ]], dtype=float32)

In [0]:
# ovako imam listu embeddinga, za svaki batch zasebno! bolje
predicted_embeddings = []

i = 0
for batch_file in listdir_sorted:
  f = h5py.File("/content/drive/My Drive/hdf5"+"/"+batch_file)
  voxels = f['label-voxel'].value
  voxels = np.swapaxes(voxels, 1, 4)
  predicted_embeddings.append(encoder.predict_on_batch(voxels))
  print(i, end = " ")
  i += 1
  
  

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

In [0]:
len(predicted_embeddings)

101

In [0]:
predicted_embeddings = np.array(predicted_embeddings)
predicted_embeddings.shape

(101,)

In [0]:
np.save("embeddings_list.npy", predicted_embeddings)

In [0]:
np.save("embeddings.npy", embeddings)

In [0]:
from google.colab import files
files.download("embeddings_list.npy")

In [0]:
!ls

autoencoder8.h5      hdf5				phaseTwo_ImageNet.ipynb
embeddings_list.npy  phaseOne_Autoencoder.ipynb		Untitled0.ipynb
embeddings.npy	     phaseTwo_generateEmbeddings.ipynb


In [0]:
!kill

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [0]:
!kill -9 -1